# Extract phases
Created 02/06/2024

Objectives:
* Extract the fermionic and proj rep group cohomologies from the solutions calculated [here](sweep_and_find_solutions.ipynb)

# Package imports

In [1]:
import sys

In [2]:
sys.path.append("../../")

In [3]:
from itertools import chain, groupby, combinations
import re

from collections import Counter, namedtuple, defaultdict

In [4]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os
import pickle

In [5]:
import numpy as np
from jax import numpy as jnp
import pandas as pd

import matplotlib.pyplot as plt

import scipy

In [6]:
import quimb as qu
import quimb.tensor as qtn
from quimb.tensor.optimize import TNOptimizer

/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:54: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:36: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:73: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(


# Load data

## Wavefunctions

In [7]:
DATA_DIR_1 = r"../../data/interpolated_trivial_to_nontrivial_fermionic_trivial_proj_rep_200_site_dmrg/"
DATA_DIR_2 = r"../../data/interpolated_nontrivial_fermionic_proj_rep_to_nontrivial_proj_rep_200_site_dmrg/"

In [8]:
def parse_file_name(file_name):
    interpolation = int(file_name.split('_')[0])/100

    return interpolation

In [9]:
loaded_data_triv_proj_rep = dict()
energies_triv_proj_rep = dict()

for local_file_name in list(os.walk(DATA_DIR_1))[0][2]:
    f_name = r"{}/{}".format(DATA_DIR_1, local_file_name, ignore_unknown=False)

    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data_triv_proj_rep[data_info]=data['wavefunction']
        energies_triv_proj_rep[data_info]=data['energy']

In [10]:
loaded_data_non_triv_proj_rep = dict()
energies_non_triv_proj_rep = dict()

for local_file_name in list(os.walk(DATA_DIR_2))[0][2]:
    f_name = r"{}/{}".format(DATA_DIR_2, local_file_name, ignore_unknown=False)

    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data_non_triv_proj_rep[data_info]=data['wavefunction']
        energies_non_triv_proj_rep[data_info]=data['energy']

In [11]:
Counter(
    tuple(psi.get_B(i).get_leg_labels())
    for psi in loaded_data_triv_proj_rep.values()
    for i in range(psi.L)
)

Counter({('vL', 'p', 'vR'): 4200})

In [12]:
Counter(
    tuple(psi.get_B(i).get_leg_labels())
    for psi in loaded_data_non_triv_proj_rep.values()
    for i in range(psi.L)
)

Counter({('vL', 'p', 'vR'): 4200})

## Boundary operator solutions

In [13]:
SOL_DIR = r"solutions/"

In [14]:
file_name_pattern = re.compile(r'^(?:non_)?triv_\d\.\d+_\d_\d_\d+\.pickle$')

In [15]:
def parse_file_name(file_name):
    if not bool(file_name_pattern.match(file_name)):
        print(file_name)
        return None

    file_name = '.'.join((file_name.split('.'))[:-1])

    if file_name[0] == 'n':
        proj_rep=1
        b, bs, fs, i = file_name.split('_')[2:]
    elif file_name[0] == 't':
        proj_rep=0
        b, bs, fs, i = file_name.split('_')[1:]
    else:
        return None

    b = float(b)
    bs = int(bs)
    fs = int(fs)
    i = int(i)
    
    return (proj_rep, b, bs, fs, i)

In [16]:
boundary_operator_solutions = dict()

for local_file_name in list(os.walk(SOL_DIR))[0][2]:
    f_name = r"{}/{}".format(SOL_DIR, local_file_name, ignore_unknown=False)

    key = parse_file_name(local_file_name)

    if key is not None:
        with open(f_name, 'rb') as f:
            out = pickle.load(f)
    
            boundary_operator_solutions[key] = out

### Check values

In [17]:
scores = [float(v[0]._value) for v in boundary_operator_solutions.values()]

In [18]:
pd.Series(scores).describe()

count    406.000000
mean      11.015315
std       19.295213
min        0.006658
25%        0.033113
50%        0.465950
75%        5.974640
max       50.006615
dtype: float64

In [19]:
score_pairs = defaultdict(list)

for k, v in boundary_operator_solutions.items():
    score_pairs[k[:-1]].append((k[-1], float(v[0]._value)))

In [20]:
help(min)

Help on built-in function min in module builtins:

min(...)
    min(iterable, *[, default=obj, key=func]) -> value
    min(arg1, arg2, *args, *[, key=func]) -> value
    
    With a single iterable argument, return its smallest item. The
    default keyword-only argument specifies an object to return if
    the provided iterable is empty.
    With two or more arguments, return the smallest argument.



In [21]:
best_score_pairs = {
    k: min(v, key=lambda x: x[1]) for k, v in score_pairs.items()
}

In [22]:
best_scores = [v[1] for v in best_score_pairs.values()]

In [23]:
len(best_score_pairs)

252

In [24]:
pd.Series(best_scores).describe()

count    252.000000
mean       2.507410
std       10.107527
min        0.006658
25%        0.019109
50%        0.054681
75%        0.294753
max       50.004364
dtype: float64

In [25]:
[k for k, v in best_score_pairs.items() if v[1]>1]

[(0, 0.45, 3, 0),
 (1, 0.5, 3, 0),
 (1, 0.55, 2, 0),
 (1, 0.1, 2, 1),
 (1, 0.35, 2, 1),
 (1, 0.45, 1, 0),
 (1, 0.35, 2, 0),
 (1, 0.5, 3, 1),
 (0, 0.5, 3, 0),
 (1, 0.55, 1, 0),
 (1, 0.45, 3, 0),
 (1, 0.5, 2, 1),
 (1, 0.5, 1, 1),
 (1, 0.45, 1, 1),
 (1, 0.5, 1, 0),
 (0, 0.45, 1, 1),
 (1, 0.55, 2, 1),
 (1, 0.45, 2, 0),
 (1, 0.3, 3, 0),
 (1, 0.35, 3, 1),
 (1, 0.45, 3, 1),
 (0, 0.5, 1, 1),
 (1, 0.45, 2, 1),
 (0, 0.5, 3, 1),
 (0, 0.5, 1, 0),
 (1, 0.15, 1, 0),
 (1, 0.5, 2, 0),
 (1, 0.4, 3, 1),
 (0, 0.45, 3, 1),
 (1, 0.4, 1, 0),
 (0, 0.45, 1, 0),
 (1, 0.2, 3, 0),
 (1, 0.0, 2, 1),
 (1, 0.0, 3, 0),
 (1, 0.4, 2, 1),
 (1, 0.35, 3, 0)]

In [26]:
best_score_pairs[(0, 0, 1, 0)]

(1, 0.017019841820001602)

In [27]:
best_boundary_operators = {
    k: boundary_operator_solutions[(*k, v[0])]
    for k, v in best_score_pairs.items()
    if k[1] != 0.5
}

In [28]:
len(best_boundary_operators)

240

# Definitions

In [29]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [30]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [31]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [32]:
fermionic_np_symmetries = [np_I, np_JW]

In [33]:
def generate_problem_rdm(quimb_psi, symmetry_site_pairs, leftmost_symmetry_site,
                         num_symmetry_sites, num_boundary_sites):
    q_top = quimb_psi.copy(deep=True)
    for i, s in symmetry_site_pairs:
        q_top.gate(
            s,
            where=i,
            contract=False,
            inplace=True
        )

    
    indices_to_map = list(chain(
        range(leftmost_symmetry_site-num_boundary_sites, leftmost_symmetry_site),
        range(leftmost_symmetry_site+num_symmetry_sites, leftmost_symmetry_site+num_symmetry_sites+num_boundary_sites)
    ))

    index_mapping = {f'k{i}': f'b{i}' for i in indices_to_map}

    q_bottom = (
        quimb_psi
        .copy()
        .reindex(index_mapping, inplace=True)
        .conj()
    )

    sites_to_contract = {
        'left': list(range(leftmost_symmetry_site-num_boundary_sites)),
        'middle': list(range(leftmost_symmetry_site, leftmost_symmetry_site+num_symmetry_sites)),
        'right': list(range(leftmost_symmetry_site+num_symmetry_sites+num_boundary_sites, quimb_psi.L))
    }

    tags_to_contract = {
        k: [f'I{i}' for i in v]
        for k, v in sites_to_contract.items()
    }

    tn = (q_top & q_bottom)

    tnc = (
        tn
        .contract(tags_to_contract['left'])
        .contract(tags_to_contract['middle'])
        .contract(tags_to_contract['right'])
    )

    return tnc

In [34]:
def generate_rdm_from_tenpy_psi(mps_psi, symmetry_site_pairs,
    leftmost_symmetry_site, num_symmetry_sites, num_boundary_sites):
    
    psi_arrays = list()
    psi_arrays.append(mps_psi.get_B(0, 'Th')[0, ...].to_ndarray())
    for i in range(1, mps_psi.L-1):
        psi_arrays.append(mps_psi.get_B(i).to_ndarray())
    psi_arrays.append(mps_psi.get_B(mps_psi.L-1)[..., 0].to_ndarray())
    
    q1 = (
        qtn
        .tensor_1d
        .MatrixProductState(psi_arrays, shape='lpr')
    )
    
    problem_rdm = generate_problem_rdm(
        q1,
        symmetry_site_pairs,
        left_most_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    return problem_rdm

## Optimisation functions

In [35]:
def split_mpo_pair(mpo_pair):
    ml = qtn.TensorNetwork(
        list(map(mpo_pair.tensor_map.__getitem__, mpo_pair.tag_map['left_mpo']))
    )

    mr = qtn.TensorNetwork(
        list(map(mpo_pair.tensor_map.__getitem__, mpo_pair.tag_map['right_mpo']))
    )

    return (ml, mr)

In [36]:
def overlap_loss_function(ml, mr, rdm_tn, epsilon=0):
    c = (rdm_tn & ml & mr) ^ ...

    c_abs_squared = (
        c
        *jnp.conjugate(c)
    )
    #c_abs_squared = c_abs_squared.astype('float32')
    c_abs = (jnp.sqrt(c_abs_squared+epsilon))

    target = jnp.sqrt(1+epsilon)
    loss = (c_abs - target)**2

    return loss

In [37]:
def overlap_loss_function_mpo_pair(mpo_pair, rdm_tn):
    ml, mr = split_mpo_pair(mpo_pair)

    return overlap_loss_function(ml, mr, rdm_tn)

In [38]:
regex_s = r"^I\d+$"
regex_p = re.compile(regex_s)

In [39]:
def relabel_mpo(mpo, k_label, b_label):
    site_locs = [
        int(k[1:]) for k in mpo.tag_map
        if bool(re.search(regex_p, k))
    ]

    k_in_indices = [f'k{i}' for i in site_locs]
    j_in_indices = [f'b{i}' for i in site_locs]

    k_out_indices = [f'{k_label}{i}' for i in site_locs]
    j_out_indices = [f'{b_label}{i}' for i in site_locs]

    mapping = dict(
        chain(
            zip(k_in_indices, k_out_indices),
            zip(j_in_indices, j_out_indices)
        )
    )

    mpo.reindex(mapping, inplace=True)

In [40]:
def unitarity_tn(tn, total_physical_dim):
    ms = [tn.copy(), tn.copy(), tn.copy()]

    relabel_mpo(ms[0], 'k', 'l')
    relabel_mpo(ms[1], 'm', 'l')
    relabel_mpo(ms[2], 'm', 'b')

    ms[0] = ms[0].conj()
    ms[2] = ms[2].conj()

    n2tn = (tn & tn.conj())
    n2 = n2tn.contract(n2tn.tag_map)
    n4tn = (tn & ms[0] & ms[1] & ms[2])
    n4 = n4tn.contract(n4tn.tag_map)

    out = jnp.real(n4 - 2*n2 + total_physical_dim)

    return out

In [41]:
def overall_loss_function(mpo_pair, rdm_tn, total_physical_dimension,
    unitary_cost_coefficient=1, overlap_cost_coefficient=1, losses=None):
    ml, mr = split_mpo_pair(mpo_pair)

    o_loss = overlap_loss_function(ml, mr, rdm_tn)
    ul_loss = unitarity_tn(ml, total_physical_dimension)
    ur_loss = unitarity_tn(mr, total_physical_dimension)

    out = (
        unitary_cost_coefficient*(ul_loss+ur_loss)
        + overlap_cost_coefficient*o_loss
    )

    out = jnp.real(out)

    if losses is not None:
        losses.append((o_loss, ul_loss, ur_loss))
    return out

## Extract SPT phase functions

In [42]:
def get_right_fp_overlap(rdm, mpo_l, mpo_r, leftmost_symmetry_site,
                     num_symmetry_sites, num_boundary_sites):

    tn = rdm.copy(deep=True)

    fermionic_site_indices = [
        i
        for i in range(
            leftmost_symmetry_site+num_symmetry_sites,
            leftmost_symmetry_site+num_symmetry_sites+num_boundary_sites
        )
        if i % 2 == 1
    ]

    top_fermionic_tensors = [
        (f'k{i}', next(map(tn.tensor_map.__getitem__, tn.ind_map[f'k{i}'])))
        for i in fermionic_site_indices
    ]

    bottom_fermionic_tensors = [
        (f'b{i}', next(map(tn.tensor_map.__getitem__, tn.ind_map[f'b{i}'])))
        for i in fermionic_site_indices
    ]

    for ind, t in top_fermionic_tensors:
        t.gate(
            np_JW,
            ind=ind,
            inplace=True
        )

    for ind, t in bottom_fermionic_tensors:
        t.gate(
            np_JW,
            ind=ind,
            transposed=True,
            inplace=True
        )

    out = (tn & mpo_l & mpo_r) ^ ...

    return out

In [43]:
def get_left_fp_overlap(rdm, mpo_l, mpo_r, leftmost_symmetry_site,
                     num_symmetry_sites, num_boundary_sites):

    tn = rdm.copy(deep=True)

    fermionic_site_indices = [
        i
        for i in range(
            leftmost_symmetry_site-num_boundary_sites,
            leftmost_symmetry_site
        )
        if i % 2 == 1
    ]

    top_fermionic_tensors = [
        (f'k{i}', next(map(tn.tensor_map.__getitem__, tn.ind_map[f'k{i}'])))
        for i in fermionic_site_indices
    ]

    bottom_fermionic_tensors = [
        (f'b{i}', next(map(tn.tensor_map.__getitem__, tn.ind_map[f'b{i}'])))
        for i in fermionic_site_indices
    ]

    for ind, t in top_fermionic_tensors:
        t.gate(
            np_JW,
            ind=ind,
            inplace=True
        )

    for ind, t in bottom_fermionic_tensors:
        t.gate(
            np_JW,
            ind=ind,
            transposed=True,
            inplace=True
        )

    out = (tn & mpo_l & mpo_r) ^ ...

    return out

In [44]:
def get_fp_charges(rdm, ml, mr, left_most_symmetry_site, num_symmetry_sites,
               num_boundary_sites):
    base_overlap = (rdm & ml & mr) ^ ...

    right_fp_overlap = get_right_fp_overlap(
        rdm,
        ml,
        mr,
        left_most_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    left_fp_overlap = get_left_fp_overlap(
        rdm,
        ml,
        mr,
        left_most_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    right_fp_charge = right_fp_overlap/base_overlap
    left_fp_charge = left_fp_overlap/base_overlap

    return (left_fp_charge, right_fp_charge, base_overlap)

# Extract SPT phase
## Fermionic group cohomology

In [59]:
num_boundary_sites=6
left_most_symmetry_site=60
num_symmetry_sites=80
bond_dimension=6

total_physical_dim = 2**9

In [60]:
fp_charges = dict()

for k, ops in best_boundary_operators.items():
    ml, mr = split_mpo_pair(ops[1])

    if k[0] == 0:
        tenpy_psi = loaded_data_triv_proj_rep[k[1]]
    elif k[0] == 1:
        tenpy_psi = loaded_data_non_triv_proj_rep[k[1]]

    bs = bosonic_np_symmetries[k[2]]
    fs = fermionic_np_symmetries[k[3]]

    symmetry_site_pairs = (
        [(i, bs) for i in range(left_most_symmetry_site, left_most_symmetry_site+num_symmetry_sites, 2)]
        + [(i, fs) for i in range(left_most_symmetry_site+1, left_most_symmetry_site+num_symmetry_sites+1, 2)]
    )

    rdm = generate_rdm_from_tenpy_psi(
        tenpy_psi,
        symmetry_site_pairs,
        left_most_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    out = get_fp_charges(
        rdm,
        ml,
        mr,
        left_most_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    fp_charges[k] = out

In [47]:
X = np.array(list(fp_charges.values()))

In [48]:
Counter(np.real(np.round(X[:, 1], 2)))

Counter({-1.0: 120, 1.0: 120})

In [49]:
Counter(np.real(np.round(X[:, 0], 2)))

Counter({-1.0: 120, 1.0: 120})

In [50]:
pd.Series(np.abs(X[:, 2])).describe()

count    240.000000
mean       0.949361
std        0.119625
min        0.000776
25%        0.955139
50%        0.981948
75%        0.991472
max        1.002534
dtype: float64

In [51]:
np.all(np.real(np.round(X[:, 1], 2)==np.real(np.round(X[:, 0], 2))))

True

In [52]:
single_fp_charges = {k: np.real(np.round(v[0], 2)) for k, v in fp_charges.items()}

In [53]:
g = groupby(single_fp_charges.items(), key=lambda x: x[0][:-1])
single_fp_charges_deduplicated = dict()

for k, v in g:
    values = [v1[1] for v1 in v]
    assert len(Counter(values)) == 1
    single_fp_charges_deduplicated[k] = values[0]

In [54]:
single_fp_charges_deduplicated

{(0, 0.45, 3): -1.0,
 (0, 0.15, 1): -1.0,
 (1, 0.55, 2): 1.0,
 (0, 0.95, 1): 1.0,
 (1, 0.7, 3): -1.0,
 (1, 0.1, 2): -1.0,
 (1, 0.1, 1): 1.0,
 (0, 0.9, 2): 1.0,
 (1, 0.65, 2): 1.0,
 (0, 0.3, 3): -1.0,
 (1, 0.2, 1): 1.0,
 (1, 0.35, 2): -1.0,
 (0, 0.4, 2): 1.0,
 (1, 0.25, 2): -1.0,
 (1, 0.45, 1): 1.0,
 (1, 0.75, 1): -1.0,
 (1, 1.0, 1): -1.0,
 (1, 0.0, 2): -1.0,
 (1, 0.55, 1): -1.0,
 (1, 0.1, 3): -1.0,
 (1, 0.2, 2): -1.0,
 (0, 0.65, 3): 1.0,
 (1, 0.45, 3): -1.0,
 (1, 0.05, 3): -1.0,
 (0, 0.7, 2): 1.0,
 (1, 0.4, 1): 1.0,
 (0, 0.95, 2): 1.0,
 (0, 0.75, 1): 1.0,
 (0, 0.2, 2): 1.0,
 (0, 0.85, 2): 1.0,
 (1, 0.8, 3): -1.0,
 (1, 0.05, 2): -1.0,
 (0, 0.45, 1): -1.0,
 (0, 0.65, 1): 1.0,
 (1, 0.25, 1): 1.0,
 (0, 0.0, 1): -1.0,
 (1, 1.0, 2): 1.0,
 (1, 0.8, 1): -1.0,
 (1, 0.95, 2): 1.0,
 (1, 0.3, 2): -1.0,
 (1, 0.45, 2): -1.0,
 (1, 0.3, 3): -1.0,
 (1, 0.7, 2): 1.0,
 (1, 0.35, 3): -1.0,
 (1, 0.9, 3): -1.0,
 (0, 0.75, 2): 1.0,
 (0, 0.55, 3): 1.0,
 (1, 0.0, 1): 1.0,
 (0, 0.3, 2): 1.0,
 (0, 0.1, 1): -1.0,

Pull out correct cases.

In [55]:
signs_to_01 = {1.0: 0, -1.0: 1}

In [56]:
correct_boundary_operators = {
    k: best_boundary_operators[(*k, signs_to_01[v])][1]
    for k, v in single_fp_charges_deduplicated.items()
}

In [57]:
len(correct_boundary_operators)

120

In [58]:
correct_boundary_operators

{(0, 0.45, 3): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (0, 0.15, 1): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (1, 0.55, 2): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (0, 0.95, 1): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (1, 0.7, 3): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (1, 0.1, 2): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (1, 0.1, 1): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (0, 0.9, 2): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (1, 0.65, 2): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (0, 0.3, 3): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (1, 0.2, 1): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (1, 0.35, 2): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (0, 0.4, 2): TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6),
 (1, 0.25, 2): TensorNetwork1D(tensors=12, indices=34, L=6

## Proj rep group cohomology


### Group data definitions

In [61]:
symmetry_labels = [
    'II',
    'IX',
    'XI',
    'XX'
]

In [62]:
symmetry_combination_labels = list(combinations(symmetry_labels, 2))

In [63]:
symmetry_combination_labels

[('II', 'IX'),
 ('II', 'XI'),
 ('II', 'XX'),
 ('IX', 'XI'),
 ('IX', 'XX'),
 ('XI', 'XX')]

In [64]:
e, a, b, c = symmetry_labels

In [65]:
group_products = [
    (a, b, c),
    (b, a, c),
    (a, c, b),
    (c, a, b),
    (c, b, a),
    (b, c, a)
]

### Evaluate

In [70]:
def tensor_network_to_mpo(tn):
    sites = [
        int(s[1:]) for s in tn.tag_map
        if s[0]=='I'
    ]
    
    rho = qtn.MatrixProductOperator(
        [t.data for t in tn.tensors],
        sites=sites
    )

    return rho

In [81]:
def get_proj_rep_phase(rho, u_g, u_h, u_gh):
    t = (
        rho
        ._apply_mpo(qu.dag(u_g), compress=False, contract=False)
        ._apply_mpo(qu.dag(u_h), compress=False, contract=False)
        ._apply_mpo(u_gh, compress=False, contract=False)
    )

    out = t.trace()

    return out/(np.abs(out))

In [82]:
def get_quimb_mps_from_tenpy_mps(tenpy_mps):
    psi_arrays = list()
    psi_arrays.append(tenpy_mps.get_B(0, 'Th')[0, ...].to_ndarray())
    for i in range(1, tenpy_mps.L-1):
        psi_arrays.append(tenpy_mps.get_B(i).to_ndarray())
    psi_arrays.append(tenpy_mps.get_B(tenpy_mps.L-1)[..., 0].to_ndarray())
    
    quimb_mps = (
        qtn
        .tensor_1d
        .MatrixProductState(psi_arrays, shape='lpr')
    )

    return quimb_mps

In [108]:
def get_proj_rep_phases(tenpy_mps, unitaries):
    quimb_mps = get_quimb_mps_from_tenpy_mps(tenpy_mps)
    sites = [
        int(s[1:]) for s in unitaries[0].tag_map
        if s[0]=='I'
    ]
    rho = quimb_mps.partial_trace_to_mpo(sites, rescale_sites=False)
    out = list()

    unitaries_dict = {
        k: v for k, v in zip(symmetry_labels[1:], unitaries)
    }

    out_phases = list()
    for a, b, c in group_products:
        phase = get_proj_rep_phase(
            rho,
            unitaries_dict[a],
            unitaries_dict[b],
            unitaries_dict[c]
        )

        out_phases.append(phase)

    return out_phases

In [109]:
interpolation_values = np.delete(np.round(np.linspace(0, 1, 21), 3), 10)

In [110]:
interpolation_values

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

In [111]:
triv_proj_rep_phases = list()

for t in interpolation_values:
    tenpy_mps = loaded_data_triv_proj_rep[t]

    unitaries = [correct_boundary_operators[(0, t, i)] for i in [1,2,3]]

    unitary_pairs = [split_mpo_pair(p) for p in unitaries]
    mpo_pairs = [(tensor_network_to_mpo(x), tensor_network_to_mpo(y)) for x,y in unitary_pairs]
    left_unitaries, right_unitaries = zip(*mpo_pairs)

    left_phases = get_proj_rep_phases(tenpy_mps, left_unitaries)
    right_phases = get_proj_rep_phases(tenpy_mps, right_unitaries)

    triv_proj_rep_phases.append([left_phases, right_phases])

In [112]:
triv_proj_rep_phases = np.array(triv_proj_rep_phases)

In [113]:
triv_proj_rep_phases.shape

(20, 2, 6)

In [114]:
np.round(
    triv_proj_rep_phases[..., ::2]/triv_proj_rep_phases[..., 1::2],
    2
)

array([[[1.-0.j  , 1.+0.j  , 1.-0.j  ],
        [1.+0.j  , 1.-0.j  , 1.+0.j  ]],

       [[1.-0.j  , 1.+0.j  , 1.+0.j  ],
        [1.+0.j  , 1.+0.j  , 1.+0.j  ]],

       [[1.+0.j  , 1.-0.j  , 1.-0.j  ],
        [1.-0.j  , 1.-0.j  , 1.-0.j  ]],

       [[1.-0.j  , 1.-0.j  , 1.-0.j  ],
        [1.-0.j  , 1.+0.j  , 1.-0.j  ]],

       [[1.-0.j  , 1.-0.j  , 1.-0.j  ],
        [1.+0.j  , 1.+0.j  , 1.+0.j  ]],

       [[1.+0.j  , 1.+0.j  , 1.+0.j  ],
        [1.+0.j  , 1.-0.01j, 1.+0.j  ]],

       [[1.-0.j  , 1.-0.j  , 1.+0.j  ],
        [1.+0.j  , 1.-0.j  , 1.+0.j  ]],

       [[1.+0.j  , 1.+0.j  , 1.-0.j  ],
        [1.+0.j  , 1.+0.02j, 1.+0.j  ]],

       [[1.+0.j  , 1.-0.j  , 1.+0.j  ],
        [1.-0.01j, 1.-0.01j, 1.+0.01j]],

       [[1.-0.j  , 1.+0.04j, 1.-0.j  ],
        [1.-0.j  , 1.+0.01j, 1.-0.j  ]],

       [[1.+0.03j, 1.-0.03j, 1.-0.01j],
        [1.-0.02j, 1.+0.04j, 1.+0.01j]],

       [[1.-0.02j, 1.-0.01j, 1.+0.j  ],
        [1.-0.03j, 1.-0.01j, 1.+0.01j]],

       [[1.+0.01

In [115]:
non_triv_proj_rep_phases = list()

for t in interpolation_values:
    tenpy_mps = loaded_data_non_triv_proj_rep[t]

    unitaries = [correct_boundary_operators[(1, t, i)] for i in [1,2,3]]

    unitary_pairs = [split_mpo_pair(p) for p in unitaries]
    mpo_pairs = [(tensor_network_to_mpo(x), tensor_network_to_mpo(y)) for x,y in unitary_pairs]
    left_unitaries, right_unitaries = zip(*mpo_pairs)

    left_phases = get_proj_rep_phases(tenpy_mps, left_unitaries)
    right_phases = get_proj_rep_phases(tenpy_mps, right_unitaries)

    non_triv_proj_rep_phases.append([left_phases, right_phases])

In [116]:
non_triv_proj_rep_phases = np.array(non_triv_proj_rep_phases)

In [117]:
non_triv_proj_rep_phases.shape

(20, 2, 6)

In [118]:
np.round(
    non_triv_proj_rep_phases[..., ::2]/non_triv_proj_rep_phases[..., 1::2],
    2
)

array([[[ 0.97+0.24j,  0.54+0.84j, -0.73+0.68j],
        [-1.  -0.01j,  1.  +0.03j,  0.63+0.77j]],

       [[-0.02-1.j  , -1.  -0.03j, -0.89+0.46j],
        [ 0.89-0.46j, -0.98+0.2j , -0.96+0.27j]],

       [[-0.49+0.87j,  0.98+0.19j, -0.87+0.49j],
        [-0.54-0.84j,  0.09-1.j  , -1.  -0.05j]],

       [[ 0.05-1.j  ,  0.89+0.45j, -0.9 -0.43j],
        [-0.46-0.89j,  0.14+0.99j, -0.96+0.27j]],

       [[-0.99-0.14j,  0.89+0.46j,  0.95+0.3j ],
        [ 0.9 -0.43j,  0.46-0.89j,  0.32+0.95j]],

       [[-0.74-0.67j,  0.89-0.45j, -0.84-0.54j],
        [-0.9 -0.43j, -1.  -0.05j, -0.6 +0.8j ]],

       [[-0.83-0.56j, -0.93+0.37j, -0.78+0.62j],
        [-0.98+0.18j, -1.  -0.01j, -0.65+0.76j]],

       [[-1.  +0.j  , -0.47+0.89j, -0.54-0.84j],
        [-0.98-0.19j, -0.99-0.13j,  0.87+0.49j]],

       [[-1.  +0.08j,  0.98-0.2j ,  0.4 +0.92j],
        [ 0.02-1.j  , -0.82-0.57j,  0.97+0.26j]],

       [[-0.88+0.48j,  1.  -0.04j,  1.  +0.1j ],
        [ 0.37-0.93j, -1.  +0.08j, -1.  +0.07j]],



# Conclusion
Working, except for the non-trivial proj rep case...